In [148]:
# Import libraries and dependencies

import pandas as pd
from pathlib import Path
import numpy as np
from sklearn.linear_model import LinearRegression
import hvplot.pandas

In [105]:
# Read in the data 
# The first row in the data gives the soruce, so we need to skip over it
eth = pd.read_csv(Path('../Resources/Gemini_ETHUSD_1h.csv'),
                 skiprows=1,
                  index_col='date',
                  parse_dates=True,
                  infer_datetime_format=True)

# Display data
display(eth.head())
display(eth.tail())

,unix,symbol,open,high,low,close,Volume ETH,Volume USD
date,,,,,,,,
2022-10-13 00:00:00,1665619200000,ETH/USD,1294.40,1296.60,1292.46,1295.26,162.209379,210103.320244
2022-10-12 23:00:00,1665615600000,ETH/USD,1294.70,1296.29,1292.87,1294.40,119.920839,155225.534002
2022-10-12 22:00:00,1665612000000,ETH/USD,1299.77,1301.22,1294.44,1294.70,158.825518,205631.398155
2022-10-12 21:00:00,1665608400000,ETH/USD,1298.96,1303.00,1298.77,1299.77,168.917546,219553.958764
2022-10-12 20:00:00,1665604800000,ETH/USD,1297.90,1299.41,1297.01,1298.96,21.727735,28223.458656


,unix,symbol,open,high,low,close,Volume ETH,Volume USD
date,,,,,,,,
2016-05-09 17:00:00,1462813200,ETH/USD,9.83,9.83,9.48,9.49,329.553213,3127.459991
2016-05-09 16:00:00,1462809600,ETH/USD,9.99,9.99,9.79,9.83,62.379450,613.189994
2016-05-09 15:00:00,1462806000,ETH/USD,10.00,10.00,9.99,9.99,10.973567,109.625935
2016-05-09 14:00:00,1462802400,ETH/USD,9.55,10.00,9.55,10.00,235.774075,2357.740748
2016-05-09 13:00:00,1462798800,ETH/USD,0.00,12.00,0.00,9.55,432.562115,4130.968200


In [106]:
# Data Cleaning
 
# Remove all data except closing price and volume
eth_prices = eth[['close', 'Volume USD']]

# Reverse the order
eth_prices = eth_prices.sort_index(ascending=True)

display(eth_prices.head())


,close,Volume USD
date,,
2016-05-09 13:00:00,9.55,4130.968200
2016-05-09 14:00:00,10.00,2357.740748
2016-05-09 15:00:00,9.99,109.625935
2016-05-09 16:00:00,9.83,613.189994
2016-05-09 17:00:00,9.49,3127.459991


In [107]:
# Calculate hourly percent change (this will be the response variable) 
eth_prices['hourly_returns'] = eth_prices['close'].pct_change()
eth_prices.head()


,close,Volume USD,hourly_returns
date,,,
2016-05-09 13:00:00,9.55,4130.968200,NaN
2016-05-09 14:00:00,10.00,2357.740748,0.047120
2016-05-09 15:00:00,9.99,109.625935,-0.001000
2016-05-09 16:00:00,9.83,613.189994,-0.016016
2016-05-09 17:00:00,9.49,3127.459991,-0.034588


In [110]:
# For the regression, we need each row to have the following values (predictors):
# 1. The 20-hour lagged hourly return
# 2. The 24-hour lagged hourly return
# 3. The 28-hour lagged hourly return
# 4. The current month
# 5. The current day
# 6. The current year

# Remove NA rows
eth_prices = eth_prices.dropna()

# Set date/month/year
eth_prices['year'] = eth_prices.index.year
eth_prices['month'] = eth_prices.index.month
eth_prices['day'] = eth_prices.index.day
eth_prices.head()

,close,Volume USD,hourly_returns,month,day,year
date,,,,,,
2016-05-09 14:00:00,10.00,2357.740748,0.047120,5,9,2016
2016-05-09 15:00:00,9.99,109.625935,-0.001000,5,9,2016
2016-05-09 16:00:00,9.83,613.189994,-0.016016,5,9,2016
2016-05-09 17:00:00,9.49,3127.459991,-0.034588,5,9,2016
2016-05-09 18:00:00,9.36,495.651312,-0.013699,5,9,2016


Notice that some of the hours had no trading volume. Looking at how this data varies by year, it becomes clear that almost all of the "zero-volume days" happened before 2018. It seems likely that in the early days of Ethereum, when trading volume was limited, there were more discrepencies in the hourly data. Another explanation is cases where the exchange halted trading for extended periods of time. In light of this, we will remove these items from the dataset and any rows that reference these rows.

In [125]:
sum(eth_prices['Volume USD'] == 0)
zero_vol = eth_prices[eth_prices['Volume USD'] == 0]

zero_vol.groupby('year').count().sum()

close             2485
Volume USD        2485
hourly_returns       0
month             2485
day               2485
lag20             1587
lag24             1490
lag28             1622
dtype: int64

In [119]:
eth_prices['hourly_returns'] = np.where(eth_prices['Volume USD'] == 0, np.nan, eth_prices['hourly_returns']

In [120]:
# Calculate lagged hourly returns
eth_prices['lag20'] = eth_prices['hourly_returns'].shift(20)
eth_prices['lag24'] = eth_prices['hourly_returns'].shift(24)
eth_prices['lag28'] = eth_prices['hourly_returns'].shift(28)

In [128]:
# Remove values that reference hours with zero volume
eth_prices = eth_prices.dropna()

50419

In [151]:

# Do the model
eth_model = LinearRegression()
X = eth_prices[['lag20', 'lag24', 'lag28', 'month', 'day', 'year']]
Y = eth_prices['hourly_returns']
model.fit(X, Y)
Y_pred = model.predict(X)

model.score(X, Y)

0.0012396077833867203

TypeError: '>=' not supported between instances of 'Version' and 'str'

:BoxWhisker   [Variable]   (value)